In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
import copy
import open3d as o3d

In [ ]:
def get_points(s):
    pattern = re.compile(r'user_message\|([0-9\.\,\-e]*,)')
    new_points = []
    for i in re.findall(pattern, s):
        i = i.split(',')[:-1]
        if len(i) == 0:
            continue
        new_points.append([float(x) for x in i])
    new_points = np.array(new_points)
    return new_points

In [ ]:
def visualize_pointcloud(pcd, show_draw=False):
    pcd = copy.deepcopy(pcd)
    o3d.visualization.draw_plotly([pcd])
    if show_draw:
        R = pcd.get_rotation_matrix_from_xyz((-np.pi / 2, 0, np.pi / 2))
        pcd.rotate(R, center=(0, 0, 0))
        o3d.visualization.draw([pcd], show_ui=True)

In [ ]:
with open('points.txt') as f:
    s = f.read()

In [ ]:
points = get_points(s)
points[:, 1] *= -1

pcl = o3d.geometry.PointCloud()
pcl.points = o3d.utility.Vector3dVector(points)

visualize_pointcloud(pcl)

In [ ]:
def binary_array_to_points(binary_array):
    points = []
    for z in range(len(binary_array)):
        for y in range(len(binary_array[z])):
            for x in range(len(binary_array[z][y])):
                if binary_array[z][y][x] == 1:
                    points.append((x, y, z))
    return points

In [ ]:
def point_cloud_to_2d_occupancy_map(points, start, scale, grid_size):
    """
    Convert point cloud to a 3d occupancy map

    Parameters
    ----------
    points : list of points
        [(x, y, z), ...]
    start : "bottom left" of the occupancy map
        (x, y, z)
    scale : how each dimension is scaled
        (x, y, z)
    grid_size : size of grid to be returned
        (x, y, z)

    Returns
    -------
    3d binary grid
    """

    binary_grid = np.zeros(grid_size, dtype=float)
    for point in points:
        x, y = point
        grid_x = int((x - start[0]) / scale[0])
        grid_y = int((y - start[1]) / scale[1])
        if 0 <= grid_x < grid_size[0] and 0 <= grid_y < grid_size[1]:
            binary_grid[grid_x, grid_y] = 1

    binary_grid = binary_grid.T

    return binary_grid

def point_cloud_to_3d_occupancy_map(points, start, scale, grid_size):
    """
    Convert point cloud to a 3d occupancy map

    Parameters
    ----------
    points : list of points
        [(x, y, z), ...]
    start : "bottom left" of the occupancy map
        (x, y, z)
    scale : how each dimension is scaled
        (x, y, z)
    grid_size : size of grid to be returned
        (x, y, z)

    Returns
    -------
    3d binary grid
    """

    grid = []
    for i in range(grid_size[2]):
        z_min = start[2] + scale[2] * i
        z_max = start[2] + scale[2] * (i + 1)

        mask = (points[:, 2] >= z_min) & (points[:, 2] < z_max)
        flat_points = points[mask, :2]

        new_grid = point_cloud_to_2d_occupancy_map(flat_points, start[:2], scale[:2], grid_size[:2])
        grid.append(new_grid)

    return grid

In [ ]:
def calc_real_to_array(position, start, scale):
    """
    Go from real world to numpy array
    input:
        position: (x, y, z) [m]
    output:
        array_position: (x, y, z) [grid]
    """
    return tuple(int((position[i] - start[i]) / scale[i]) for i in range(len(start)))

In [ ]:
def point_cloud_to_3d_occupancy_map(points, start, scale, grid_size):
    """
    Convert point cloud to a 3d occupancy map

    Parameters
    ----------
    points : list of points
        [(x, y, z), ...]
    start : "bottom left" of the occupancy map
        (x, y, z)
    scale : how each dimension is scaled
        (x, y, z)
    grid_size : size of grid to be returned
        (x, y, z)

    Returns
    -------
    3d binary grid
    """

    grid = np.zeros(tuple(reversed(grid_size)))
    converted_points = [calc_real_to_array(point, start, scale) for point in points]

    # Define the range
    min_range = (0, 0, 0)
    max_range = grid_size

    # Filter tuples within the range
    converted_points = [(x, y, z) for x, y, z in converted_points
                    if min_range[0] <= x < max_range[0]
                    and min_range[1] <= y < max_range[1]
                    and min_range[2] <= z < max_range[2]]
    
    x, y, z = zip(*converted_points)
    grid[z, y, x] = 1

    return grid

In [ ]:
point = calc_real_to_array((1.29, -0.6, -1.86), start, scale)
point

In [ ]:
calc_array_to_real(point, start, scale)

In [ ]:
grid = []

start = (-100, -100, -2)
scale = (0.5, 0.5, 0.5)
grid_size = (400, 400, 50)

grid = point_cloud_to_3d_occupancy_map(points, start, scale, grid_size)


pos = binary_array_to_points(grid)
#pos = [(p[0], -p[1], p[2]) for p in pos]

# Plot the points where the binary array is 1
pcl = o3d.geometry.PointCloud()
pcl.points = o3d.utility.Vector3dVector(pos)

visualize_pointcloud(pcl, False)

In [ ]:
def calc_array_to_real(position, start, scale):
    """
    Go from numpy array to real world
    input:
        position: (x, y, z) [grid]
    output:
        real_position: (x, y, z) [m]
    """

    return tuple(scale[i] * position[i] + start[i] for i in range(len(start)))

In [ ]:
recreated_points = [calc_array_to_real(point, start, scale) for point in pos]

In [ ]:
pcl = o3d.geometry.PointCloud()
pcl.points = o3d.utility.Vector3dVector(recreated_points)

visualize_pointcloud(pcl)